# <center>ECON 725: Computer Programming and Data Management in Economics <a class="tocSkip"></center>    
# <center>In-class Assessment #3: Intro to SQL <a class="tocSkip"></center>

## In-class Assessment <a class="tocSkip">
<hr>

Name: FirstName LastName <br>

Instructions:
1. Start with Question 0 to set up your database
2. Then answer Questions 1-5
3. Test your code before submitting

In [2]:
# make sure you have the following libraries installed in your machine!
import sqlite3
from faker import Faker
import random

### Question 0

Using the Faker library and SQLite, create a company database with the following specifications:

1. Create a database called 'company.db' with three tables:
   
   a) employees table with columns:
      - `emp_id` (INTEGER PRIMARY KEY)
      - `first_name` (TEXT)
      - `last_name` (TEXT)
      - `email` (TEXT)
      - `department_id` (INTEGER)
      - `salary` (INTEGER)
      - `hire_date` (TEXT)
   
   b) departments table with columns:
      - `dept_id` (INTEGER PRIMARY KEY)
      - `dept_name` (TEXT)
      - `location` (TEXT)
   
   c) projects table with columns:
      - `project_id` (INTEGER PRIMARY KEY)
      - `project_name` (TEXT)
      - `emp_id` (INTEGER)
      - `budget` (INTEGER)
      - `status` (TEXT)

2. Populate the tables with synthetic data:
   - Create 5 departments: 
        `(1, 'Engineering', 'San Francisco'),
        (2, 'Marketing', 'New York'),
        (3, 'Sales', 'Chicago'),
        (4, 'HR', 'Boston'),
        (5, 'Finance', 'Seattle')`
   - Generate 50 employees using Faker with:
     * Random first and last names
     * Email format: firstname.lastname@econ725.com (lowercase)
     * Random `department_id` between 1-5
     * Random salary between 50,000 and 150,000
     * Random `hire_date` within the last 5 years
   - Generate 30 projects using Faker with:
     * Project names from `fake.catch_phrase()`
     * Random emp_id between 1-50
     * Random budget between 10,000 and 500,000
     * Random status from: 'Active', 'Completed', 'On Hold', 'Cancelled'

3. Set random seeds for reproducibility:
   - `Faker.seed(42)`
   - `random.seed(42)`

4. Your code should:
   - Drop existing tables if they exist
   - Create the three tables
   - Insert all data
   - Commit the changes
   - Print confirmation message

Complete the `setup_database()` function below:

In [3]:
def setup_database():
    """Create and populate the company database"""
    # Set seeds for reproducibility
    Faker.seed(42)
    random.seed(42)
    
    # Initialize Faker
    fake = Faker()
    
    # Create database and connection
    conn = sqlite3.connect('company.db')
    cursor = conn.cursor()
    
    # Drop tables if they exist
    cursor.execute("DROP TABLE IF EXISTS employees")
    cursor.execute("DROP TABLE IF EXISTS departments")
    cursor.execute("DROP TABLE IF EXISTS projects")
    
    # Create tables
    cursor.execute("""
    CREATE TABLE employees (
        emp_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        email TEXT,
        department_id INTEGER,
        salary INTEGER,
        hire_date TEXT
    )
    """)
    
    cursor.execute("""
    CREATE TABLE departments (
        dept_id INTEGER PRIMARY KEY,
        dept_name TEXT,
        location TEXT
    )
    """)
    
    cursor.execute("""
    CREATE TABLE projects (
        project_id INTEGER PRIMARY KEY,
        project_name TEXT,
        emp_id INTEGER,
        budget INTEGER,
        status TEXT
    )
    """)
    
    # Generate and insert departments
    departments = [
        (1, 'Engineering', 'San Francisco'),
        (2, 'Marketing', 'New York'),
        (3, 'Sales', 'Chicago'),
        (4, 'HR', 'Boston'),
        (5, 'Finance', 'Seattle')
    ]
    cursor.executemany("INSERT INTO departments VALUES (?,?,?)", departments)
    
    # Generate 50 employees
    employees = []
    for i in range(1, 51):
        first = fake.first_name()
        last = fake.last_name()
        email = f"{first.lower()}.{last.lower()}@company.com"
        dept = random.randint(1, 5)
        salary = random.randint(50000, 150000)
        hire_date = fake.date_between(start_date='-5y', end_date='today').strftime('%Y-%m-%d')
        employees.append((i, first, last, email, dept, salary, hire_date))
    
    cursor.executemany("INSERT INTO employees VALUES (?,?,?,?,?,?,?)", employees)
    
    # Generate 30 projects
    projects = []
    statuses = ['Active', 'Completed', 'On Hold', 'Cancelled']
    for i in range(1, 31):
        project_name = fake.catch_phrase()
        emp_id = random.randint(1, 50)
        budget = random.randint(10000, 500000)
        status = random.choice(statuses)
        projects.append((i, project_name, emp_id, budget, status))
    
    cursor.executemany("INSERT INTO projects VALUES (?,?,?,?,?)", projects)
    
    # Commit changes
    conn.commit()
    
    # Print confirmation
    print("Database setup complete!")
    print(f"Created {len(employees)} employees, {len(departments)} departments, and {len(projects)} projects")
    
    # Close connection
    conn.close()
    pass

In [4]:
# Run the setup
setup_database()

Database setup complete!
Created 50 employees, 5 departments, and 30 projects


In [5]:
# ====================
# QUESTIONS 1-5
# ====================

# After completing Question 0, uncomment the lines below to create connection
conn = sqlite3.connect('company.db')
cursor = conn.cursor()

### Question 1 (20 points)
 Write a function called `get_high_earners()` that returns a list of tuples containing (first_name, last_name, salary) for all employees who earn more than $100,000, ordered by salary in descending order.

Expected output format: `[('John', 'Smith', 125000), ('Jane', 'Doe', 120000), ...]`


In [9]:
def get_high_earners():
    """Return employees earning more than $100,000"""
    cursor.execute("""
        SELECT first_name, last_name, salary 
        FROM employees 
        WHERE salary > 100000 
        ORDER BY salary DESC
    """)
    return cursor.fetchall()

### Question 2 (20 points)
Write a function called `calculate_dept_avg_salary()` that returns a list of tuples containing (dept_name, location, avg_salary) for each department. Round the average salary to 2 decimal places and order by avg_salary descending.

Expected output format: `[('Engineering', 'San Francisco', 95432.50), ...]`


In [10]:
def calculate_dept_avg_salary():
    """Calculate average salary by department"""
    cursor.execute("""
        SELECT d.dept_name, d.location, ROUND(AVG(e.salary), 2) as avg_salary
        FROM departments d
        JOIN employees e ON d.dept_id = e.department_id
        GROUP BY d.dept_id
        ORDER BY avg_salary DESC
    """)
    return cursor.fetchall()

### Question 3 (20 points)
 Write a function called `get_project_summary()` that takes a status parameter (e.g., 'Active', 'Completed') and returns:
- Total number of projects with that status
- Total budget for those projects
- Average budget for those projects (rounded to 2 decimal places)

Return as a dictionary with keys: 'count', 'total_budget', 'avg_budget'

Expected output: `{'count': 10, 'total_budget': 2500000, 'avg_budget': 250000.00}`


In [11]:
def get_project_summary(status):
    """Get summary statistics for projects by status"""
    cursor.execute("""
        SELECT COUNT(*), SUM(budget), ROUND(AVG(budget), 2)
        FROM projects
        WHERE status = :status
    """, {'status': status})
    
    result = cursor.fetchone()
    return {
        'count': result[0],
        'total_budget': result[1],
        'avg_budget': result[2]
    }

### Question 4 (20 points)
Write a function called `update_salary_by_dept()` that takes a department_id and a percentage increase (e.g., 10 for 10%) and updates all employees' salaries in that department by the given percentage. Return the number of employees whose salaries were updated.

Example: `update_salary_by_dept(1, 10)` should give everyone in dept 1 a 10% raise


In [12]:
def update_salary_by_dept(dept_id, percentage):
    """Update salaries for all employees in a department"""
    with conn:
        cursor.execute("""
            UPDATE employees 
            SET salary = salary * (1 + :pct/100.0)
            WHERE department_id = :dept_id
        """, {'dept_id': dept_id, 'pct': percentage})
    
    return cursor.rowcount

### Question 5 (20 points)
Write a function called `get_employee_project_info()` that returns information about employees and their projects. For each employee who has at least one Active project, return `(emp_id, full_name, dept_name, num_active_projects, total_active_budget)` where:
- `full_name` is "FirstName LastName"
- `num_active_projects` is the count of Active projects for that employee
- `total_active_budget` is the sum of budgets for that employee's Active projects

Order by `total_active_budget` descending. EXPLAIN EACH VALUE OF THE OUTPUT.

Expected output: `[(15, 'John Smith', 'Engineering', 2, 450000), ...]`

In [13]:
def get_employee_project_info():
    """Get employees with their active projects information"""
    cursor.execute("""
        SELECT 
            e.emp_id,
            e.first_name || ' ' || e.last_name as full_name,
            d.dept_name,
            COUNT(p.project_id) as num_active_projects,
            SUM(p.budget) as total_active_budget
        FROM employees e
        JOIN departments d ON e.department_id = d.dept_id
        JOIN projects p ON e.emp_id = p.emp_id
        WHERE p.status = 'Active'
        GROUP BY e.emp_id
        ORDER BY total_active_budget DESC
    """)
    return cursor.fetchall()

In [14]:
# ====================
# TEST YOUR FUNCTIONS
# ====================

if __name__ == "__main__":
    print("\n=== Testing Question 1 ===")
    try:
        result = get_high_earners()
        print(f"Found {len(result)} high earners")
        print("First 3:", result[:3])
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n=== Testing Question 2 ===")
    try:
        result = calculate_dept_avg_salary()
        print(f"Department averages:")
        for row in result:
            print(f"  {row}")
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n=== Testing Question 3 ===")
    try:
        result = get_project_summary('Active')
        print(f"Active projects summary: {result}")
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n=== Testing Question 4 ===")
    try:
        # Get initial average for dept 1
        cursor.execute("SELECT AVG(salary) FROM employees WHERE department_id=1")
        before = cursor.fetchone()[0]
        
        count = update_salary_by_dept(1, 10)
        
        cursor.execute("SELECT AVG(salary) FROM employees WHERE department_id=1")
        after = cursor.fetchone()[0]
        
        print(f"Updated {count} employees")
        print(f"Avg salary before: ${before:.2f}, after: ${after:.2f}")
        
        # Rollback to keep data consistent
        conn.rollback()
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n=== Testing Question 5 ===")
    try:
        result = get_employee_project_info()
        print(f"Found {len(result)} employees with active projects")
        print("Top 3:", result[:3])
    except Exception as e:
        print(f"Error: {e}")
    
    # Close connection
    conn.close()


=== Testing Question 1 ===
Found 19 high earners
First 3: [('Jeremy', 'Roberts', 149458), ('Joseph', 'Clark', 145647), ('Rita', 'Harrell', 141988)]

=== Testing Question 2 ===
Department averages:
  ('Marketing', 'New York', 102252.18)
  ('Engineering', 'San Francisco', 97688.46)
  ('Sales', 'Chicago', 95977.58)
  ('HR', 'Boston', 94323.0)
  ('Finance', 'Seattle', 82374.6)

=== Testing Question 3 ===
Active projects summary: {'count': 9, 'total_budget': 2927712, 'avg_budget': 325301.33}

=== Testing Question 4 ===
Updated 13 employees
Avg salary before: $97688.46, after: $107457.31

=== Testing Question 5 ===
Found 8 employees with active projects
Top 3: [(44, 'Marie Parker', 'Sales', 2, 619198), (33, 'Wendy Rice', 'Engineering', 1, 488479), (16, 'Jesse Garcia', 'Marketing', 1, 470329)]
